In [29]:
# RUN ONCE
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("marcozuppelli/stegoimagesdataset")
# print("Path to dataset files:", path)

In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import glob
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
def getLSBFloatBytes(filePath):
    img = Image.open(filePath).convert("RGB")
    pixels = np.array(img).flatten()
    lsb_array = pixels & 1

    usable_length = (len(lsb_array) // 8) * 8
    lsb_array = lsb_array[:usable_length]

    lsb_reshaped = lsb_array.reshape((-1, 8))
    powers = 2**np.arange(7, -1, -1)
    byte_values = np.dot(lsb_reshaped, powers).astype(np.float32)

    float_bytes = byte_values / 255.0
    return float_bytes.tolist()


In [3]:
### GETTING THE PATHS OF ALL THE IMAGES IN THE KAGGLE DATASET
basePath = '/Users/gaim/.cache/kagglehub/datasets/marcozuppelli/stegoimagesdataset/versions/2'

trainStegImgs = glob.glob(basePath + '/train/train/stego/*.png') #get all the steg image paths
trainCleanImgs = glob.glob(basePath + '/train/train/clean/*.png') #get all the clean image paths
testStegImgs = glob.glob(basePath + '/test/test/stego/*.png')
testCleanImgs = glob.glob(basePath + '/test/test/clean/*.png')

In [6]:
### GET THE RAW BYTES FOR EACH IMAGE AND THEN ADD THEM TO A PANDAS DATAFRAME

trainRows = []
for img in trainStegImgs[:1000]:
    rowInfo = {
        'imagePath' : img,
        'rawBytes' : getLSBFloatBytes(img),
        'hasSteg' : 1
    }
    trainRows.append(rowInfo)
    #print(img, "COMPLETED")


for img in trainCleanImgs[:1000]:
    rowInfo = {
        'imagePath' : img,
        'rawBytes' : getLSBFloatBytes(img),
        'hasSteg' : 0
    }
    trainRows.append(rowInfo)
    


trainDf = pd.DataFrame(trainRows)
trainDf = shuffle(trainDf, random_state=8)
display(trainDf)

,imagePath,rawBytes,hasSteg
1859,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1059,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
114,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8352941274642...",1
791,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8627451062202...",1
1813,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
...,...,...,...
986,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2941176593303...",1
133,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.1764705926179886, 0.1568627506494522, ...",1
361,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8313725590705...",1
1364,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0


In [9]:
### GET THE RAW BYTES FOR EACH IMAGE AND THEN ADD THEM TO A PANDAS DATAFRAME

testRows = []
for img in testStegImgs[:1000]:
    rowInfo = {
        'imagePath' : img,
        'rawBytes' : getLSBFloatBytes(img),
        'hasSteg' : 1
    }
    testRows.append(rowInfo)
    #print(img, "COMPLETED")


for img in testCleanImgs[:1000]:
    rowInfo = {
        'imagePath' : img,
        'rawBytes' : getLSBFloatBytes(img),
        'hasSteg' : 0
    }
    testRows.append(rowInfo)
    


testDf = pd.DataFrame(testRows)
testDf = shuffle(testDf, random_state=8)
display(testDf)

,imagePath,rawBytes,hasSteg
1859,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1059,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
114,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8117647171020...",1
791,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.38039216...",1
1813,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
...,...,...,...
986,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7725490331649...",1
133,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18431372...",1
361,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0392156876623...",1
1364,/Users/gaim/.cache/kagglehub/datasets/marcozup...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0


In [12]:
# trainDf.to_csv('trainDf.csv')

In [13]:
# testDf.to_csv('testDf.csv')

In [14]:
# trainDf = pd.read_csv('trainDf.csv')

In [15]:
# testDf = pd.read_csv('testDf.csv')

In [18]:
rawBytes = trainDf['rawBytes'].iloc[1]
hasSteg = trainDf['hasSteg'].iloc[1]

In [26]:
def prepareData(df):
    X = np.stack(df['rawBytes'].values)  # each list becomes a row
    y = np.array(df['hasSteg'], dtype=np.uint8)  # Binary labels
    return X, y

xTrain, yTrain = prepareData(trainDf)
xTest, yTest = prepareData(testDf)

In [27]:
display(xTrain.shape)

(2000, 98304)

In [28]:
model = Sequential()

# Add the input layer and first hidden layer
# Assuming the number of features (raw bytes per image) is the length of xTrain[0]
input_size = xTrain.shape[1]  # This is the length of your byte data per sample (number of features)

model.add(Dense(128, input_dim=input_size, activation='relu'))  # First hidden layer
model.add(Dropout(0.2))  # Dropout to prevent overfitting

# Add another hidden layer
model.add(Dense(64, activation='relu'))

# Add the output layer with a single unit for binary classification (0 or 1)
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model to see the structure
model.summary()

# Train the model
history = model.fit(xTrain, yTrain, epochs=10, batch_size=32, validation_data=(xTest, yTest))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(xTest, yTest)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

/Users/gaim/Documents/Work/Projects/steganography-detector/.detectorVenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │    12,583,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,591,361 (48.03 MB)

 Trainable params: 12,591,361 (48.03 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 122ms/step - accuracy: 0.5930 - loss: 4.5871 - val_accuracy: 0.5465 - val_loss: 0.8758
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.6743 - loss: 0.8824 - val_accuracy: 0.7370 - val_loss: 0.4964
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.7377 - loss: 0.4915 - val_accuracy: 0.7530 - val_loss: 0.5259
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.7925 - loss: 0.4451 - val_accuracy: 0.7180 - val_loss: 0.6061
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.7665 - loss: 0.4550 - val_accuracy: 0.7805 - val_loss: 0.4502
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.8285 - loss: 0.3824 - val_accuracy: 0.6965 - val_loss: 0.5269
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.7445 - loss: 0.4873 - val_accuracy: 0.7460 - val_loss: 0.4885
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - accuracy: 0.8202 - loss: 0.4390 - val_accuracy: 0.7550 - 